In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pickle

# Import dataset

In [2]:
data = pd.read_csv("../DataFormating/compressed_final.csv")

In [3]:
data.head()

,Away Team Goals,Away Team Name,Home Team Goals,Home Team Name,Player 1 Overall Diff,Player 2 Overall Diff,Player 3 Overall Diff,Player 4 Overall Diff,Player 5 Overall Diff,Player 6 Overall Diff,Player 7 Overall Diff,Player 8 Overall Diff,Player 9 Overall Diff,Player 10 Overall Diff,Player 11 Overall Diff,Avg Goals Diff,FIFA Rank Diff
0,1.0,Croatia,3.0,Brazil,5,4,7,5,8,9,10,9,11,12,13,2.000000,-16.0
1,3.0,Costa Rica,1.0,Uruguay,8,4,5,7,5,5,6,6,6,7,7,-1.333333,-8.0
2,0.0,Portugal,4.0,Germany,-5,-4,-3,-7,-8,-9,-10,-13,-10,-11,-11,4.000000,-3.0
3,0.0,Nigeria,0.0,Iran,0,2,0,2,0,1,0,2,1,0,2,0.000000,-11.0
4,0.0,Mexico,0.0,Brazil,11,2,5,5,8,8,11,9,11,12,12,0.666667,-13.0


# Set `X` and `y`

In [4]:
data.columns

Index(['Away Team Goals', 'Away Team Name', 'Home Team Goals',
       'Home Team Name', 'Player 1 Overall Diff', 'Player 2 Overall Diff',
       'Player 3 Overall Diff', 'Player 4 Overall Diff',
       'Player 5 Overall Diff', 'Player 6 Overall Diff',
       'Player 7 Overall Diff', 'Player 8 Overall Diff',
       'Player 9 Overall Diff', 'Player 10 Overall Diff',
       'Player 11 Overall Diff', 'Avg Goals Diff', 'FIFA Rank Diff'],
      dtype='object')

In [5]:
X = data.drop(["Away Team Goals", "Home Team Goals"], axis=1)

y = []
for i in range(len(data)):
    home_team_goals = data["Home Team Goals"][i]
    away_team_goals = data["Away Team Goals"][i]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [6]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [7]:
word_cup_teams = [
    "Egypt",
    "Morocco",
    "Nigeria",
    "Senegal",
    "Tunisia",
    "Australia",
    "IR Iran",
    "Japan",
    "Korea DPR",
    "Saudi Arabia",
    "Belgium",
    "Croatia",
    "Denmark",
    "England",
    "France",
    "Germany",
    "Iceland",
    "Poland",
    "Portugal",
    "Russia",
    "Serbia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Costa Rica",
    "Mexico",
    "Panama",
    "Argentina",
    "Brazil",
    "Colombia",
    "Peru",
    "Uruguay"
]

team_names = list(data["Home Team Name"].unique()) + list(data["Away Team Name"].unique()) + word_cup_teams

In [8]:
team_name_encoder = LabelEncoder().fit(team_names)

In [9]:
X["Home Team Name"] = team_name_encoder.transform(X["Home Team Name"])
X["Away Team Name"] = team_name_encoder.transform(X["Away Team Name"])

### Feature Selection

In [10]:
feature_names = [
    "Away Team Name",
    "Home Team Name",
    # "Overall",
    "Avg Goals Diff",
    "FIFA Rank Diff"
]
COLUMNS = []

for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

X = X[COLUMNS]

In [11]:
COLUMNS

['Away Team Name', 'Home Team Name', 'Avg Goals Diff', 'FIFA Rank Diff']

# Traning Session

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 4 columns):
Away Team Name    306 non-null int64
Home Team Name    306 non-null int64
Avg Goals Diff    306 non-null float64
FIFA Rank Diff    306 non-null float64
dtypes: float64(2), int64(2)
memory usage: 9.6 KB


In [13]:
X.describe()

,Away Team Name,Home Team Name,Avg Goals Diff,FIFA Rank Diff
count,306.000000,306.000000,306.000000,306.000000
mean,15.539216,14.366013,0.320261,-1.450980
std,10.359130,10.229338,1.611076,25.412645
min,0.000000,0.000000,-6.000000,-67.000000
25%,5.000000,5.000000,-0.458333,-13.750000
50%,17.000000,11.000000,0.250000,-3.000000
75%,24.000000,24.000000,1.000000,9.750000
max,33.000000,33.000000,6.000000,64.000000


In [15]:
rf_model = RandomForestClassifier(n_estimators=1100, max_depth=10, bootstrap=True, n_jobs=-1)

In [16]:
rf_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# Save model and encoders

In [17]:
with open("rf_model.b", "wb") as f:
    pickle.dump(rf_model, f)

with open("team_name_encoder.b", "wb") as f:
    pickle.dump(team_name_encoder, f)